# Initiation

In [1]:
from pyhive import presto
from pyhive import presto
import pandas as pd
import xgboost as xgb
def sql(query_sentence):
    cursor = presto.connect(protocol='https',
                            host="", #insert your host name/url
                            port= , #insert port
                            username='',
                            password='',
                            session_props={'query_max_stage_count': 500}).cursor()
    cursor.execute(query_sentence)
    return pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import gc

from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import joblib


# intended for use with XGBoost models or similar where preds are the predicted probabilities and dtrain is the training data.
# Its more integrated with XGBoost's internal data handling (dtrain.get_label())
def evalKS(preds, dtrain):
    labels = dtrain.get_label()
    fpr, tpr, threshold = roc_curve(labels, preds)
    KS = max(tpr-fpr)
    return('KS', KS)
    
# This function is more generic and seems to accept labels (true labels) and probs (predicted probabilities) directly as arguments.
def get_ks(labels, probs):
    fpr,tpr,_ = roc_curve(labels,probs)
    w=tpr-fpr
    ks_score=w.max()
    return 'ks',ks_score,1


def get_performance(model,data,fea,labels,title_name):
    nm = max([len(n) for n in title_name])
    
    for i in range(len(title_name)):
        features= data[i][fea]
        probs = model.predict_proba(features)[:,1]
        fpr,tpr,_ = roc_curve(data[i][labels],probs,drop_intermediate=False)
        auc_score = auc(fpr,tpr)
        w=tpr-fpr
        ks_score=w.max()
        ks_x=fpr[w.argmax()]
        ks_y=tpr[w.argmax()]
        

        print(title_name[i],' '*(nm-len(title_name[i])),' features.shape=',features.shape,
        '  auc_score={}'.format(roc_auc_score(data[i][labels],probs).round(5)),
        '  ks_score={}'.format(ks_score.round(5)))


In [2]:
data_1 = sql("""select * from source""")
data_1.rename(columns={'score_test2':'score_1'},inplace=True)
data_1.shape

(261294, 65)

In [5]:
data_2 = sql("""select * from source""")
data_2.rename(columns={'score':'score_2','main_iou_id':'iou_id'},inplace=True)
data_2.shape

(261294, 60)

In [7]:
data_3 = sql("""select * from source""")
data_3.rename(columns={'score':'score_3','main_iou_id':'iou_id'},inplace=True)
data_3.shape

(261294, 56)

In [8]:
data_1 = data_1.replace('', np.nan)
varlist = data_1.iloc[:,15:len(data_1.columns)].columns
for i in list(data_1[varlist].select_dtypes('O')):
    data_1[i] = data_1[i].astype(float)

In [9]:
data_2 = data_2.replace('', np.nan)
varlist = data_2.iloc[:,5:len(data_2.columns)-1].columns
for i in list(data_2[varlist].select_dtypes('O')):
    data_2[i] = data_2[i].astype(float)

In [10]:
data_3 = data_3.replace('', np.nan)
varlist = data_3.iloc[:,5:len(data_3.columns)-1].columns
for i in list(data_3[varlist].select_dtypes('O')):
    data_3[i] = data_3[i].astype(float)

In [13]:
pid_list=[]
for i in list(data_1):
    if 'mf' not in i:
        pid_list.append(i)

In [14]:
data_total = data_1[pid_list].merge(
    data_2[['iou_id','score_2']],on='iou_id').merge(
    data_3[['iou_id','score_3']],on='iou_id')
data_total.shape

(261294, 20)

# Sample Division

In [21]:
train_id = sql(""" select * from source """)
train_id.shape

(61748, 1)

In [22]:
valid_id = sql("""select * from source""")
valid_id.shape

(15438, 1)

In [23]:
set(train_id.main_person_uuid).intersection(set(valid_id.main_person_uuid))

set()

In [24]:
train = data_total[(data_total.main_iou_mon.between('2024-04','2024-05'))&(data_total.person_uuid.isin(train_id.main_person_uuid))]
valid = data_total[(data_total.main_iou_mon.between('2024-04','2024-05'))&(data_total.person_uuid.isin(valid_id.main_person_uuid))]
oot = data_total[~data_total.main_iou_mon.between('2024-04','2024-05')]
train.shape,valid.shape,oot.shape

((96740, 20), (24057, 20), (140497, 20))

In [25]:
96740+24057+140497

261294

# XGB-MO 

In [46]:
var_list = ['score_1','score_2','score_3']
# var_list = ['score_1','score_2']
len(var_list)

3

In [27]:
dtrain = xgb.DMatrix(train[var_list], label=train['flag'])
dvalid = xgb.DMatrix(valid[var_list], label=valid['flag'])

In [ ]:
params_xgb = {
    'booster': 'gbtree',               
    'objective': 'binary:logistic',    
    'disable_default_eval_metric': 1,  
    'gamma': 0,                        
    'max_depth': 3,                    
    'colsample_bytree':1,             
    'subsample': 0.8,                   
    'eta': 0.1,                       
    'seed': 10,                       
    'min_child_weight':200,            
    'tree_method': 'auto'            
    #'max_bin':                        
}

In [ ]:
num_boost_round = 1000                         
evallist = [(dtrain, 'train'),(dvalid, 'valid')]       
evals_result={}
xgb_mod = xgb.train(params_xgb, dtrain, num_boost_round,
                    evallist,feval=evalKS,                   
                    evals_result=evals_result,
                    maximize=True,                          
                    verbose_eval=10,                        
                    early_stopping_rounds=50                
                   )

In [ ]:
def get_performance_v2(model, data, fea, labels, title_name):
    nm = max([len(n) for n in title_name])
    
    for i in range(len(title_name)):
        features = data[i][fea]
        # Use predict to get probabilities for binary classification
        probs = model.predict(xgb.DMatrix(features))
        fpr, tpr, _ = roc_curve(data[i][labels], probs, drop_intermediate=False)
        auc_score = auc(fpr, tpr)
        w = tpr - fpr
        ks_score = w.max()
        ks_x = fpr[w.argmax()]
        ks_y = tpr[w.argmax()]
        
        print(title_name[i], ' ' * (nm - len(title_name[i])), ' features.shape=', features.shape,
              '  auc_score={}'.format(roc_auc_score(data[i][labels], probs).round(5)),
              '  ks_score={}'.format(ks_score.round(5)))

get_performance_v2(
    xgb_mod,
    [train_df_fei, test_df_fei],
    prod_features,
    'isbad',
    ['train', 'valid']
)

# Save model0 

In [ ]:
joblib.dump(xgb_mod,'model0.pkl')